In [1]:
from django.db import connection,transaction
from django.conf import settings
import pandas as pd
import sqlite3
import glob
from PIL import Image

In [22]:
db = "db.sqlite3"
connection = sqlite3.connect(db)
c = connection.cursor()
PATH_DATASET = "../model/dataset" 

### Generate data to fill database

In [3]:
kml_images = glob.glob("%s/images_located/Captura17.kml" % PATH_DATASET)
images = glob.glob("%s/images_located_burnt/Captura17.png" % PATH_DATASET)
df_region_metadata = pd.read_csv("%s/regions.csv" % PATH_DATASET)
kml_images.sort()
images.sort()

In [5]:
def convert_coordinates(coordinates):
    new_coordinates = []
    coor_0 = coordinates[0].split(',')[:2]
    coor_1 = coordinates[1].split(',')[:2]
    coor_2 = coordinates[2].split(',')[:2]
    coor_3 = coordinates[3].split(',')[:2]
    new_coordinates += [coordinates[0]]
    new_coordinates += ["%s,%s,0" % ((coor_0[0], coor_1[1])) ]
    new_coordinates += ["%s,%s,0" % ((coor_3[0], coor_1[1])) ]
    new_coordinates += ["%s,%s,0" % ((coor_3[0], coor_0[1])) ]
    return new_coordinates

In [7]:
import glob
import bs4


Image = {'gallery_images':[], 'fCorner':[], 'sCorner':[], 'tCorner':[], 'ftCorner':[], 'burntArea':[], 'img':[] }
for i in range(0, 1):
    kml = kml_images[i]
    with open(kml) as kml_file:
        page = kml_file.read()
        bs = bs4.BeautifulSoup(page, "lxml")
        coordinates = bs.document.linearring.findAll("coordinates")[0].text.lstrip().lstrip().rstrip().split(' ')[0:4]
        coordinates = convert_coordinates(coordinates)
        Image['gallery_images'] += ['Demo Burnt forest %d' % i]
        Image['fCorner'] += [coordinates[1]]
        Image['sCorner'] += [coordinates[2]]
        Image['tCorner'] += [coordinates[3]]
        Image['ftCorner'] += [coordinates[0]]
        Image['burntArea'] += [0]
        Image['img'] += ["upload_images/%s"% images[i].split('/')[-1]]

In [19]:
Image['gallery_images']= 'Demo forest'

### Generate SQL queries

In [27]:

c.execute("SELECT * FROM gsoc2019_gallery ;").fetchall()

ProgrammingError: Cannot operate on a closed database.

In [23]:
for i in range(0, 1):
    c.execute("INSERT INTO gsoc2019_gallery (name) VALUES (?);" , (Image['gallery_images'][i],))


In [25]:
for i in range(0, 1):
    c.execute("INSERT INTO gsoc2019_image VALUES (?, ?, ?, ?, ?, ?, ?, ?);" , ((12310),
                                                                                             Image['fCorner'][i],
                                                                                             Image['sCorner'][i],
                                                                                             Image['tCorner'][i],
                                                                                             Image['ftCorner'][i],
                                                                                             Image['img'][i],
                                                                                             Image['gallery_images'][i],
                                                                                             Image['burntArea'][i],))



In [13]:
c.execute("INSERT INTO gsoc2019_image VALUES (?, ?, ?, ?, ?, ?, ?, ?);" , ((100),  '0.7793753075876952,42.04689302307633,0',
    '0.7920521003639958,42.04689302307633,0',
  '0.7920521003639958,42.05406437718434,0',
  '0.7793753075876952,42.05406437718434,0',
  'upload_images/Captura13.png',
  'Demo Burnt forest 4',
                                                                           




<module 'PIL.Image' from '/Users/Marcelpv96/anaconda3/lib/python3.7/site-packages/PIL/Image.py'>

### Save the changes

In [26]:
connection.commit()
connection.close()